In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from langdetect import detect
from translate import Translator
import os

In [8]:
# translator = Translator(to_lang="en")
# import re
# def extract_reviews(soup):
#     reviews = []
#     review_divs = soup.find_all('div', {'data-hook': 'review'})

#     for div in review_divs:
#         review = {}
#         review['customer_name'] = div.find('span', {'class': 'a-profile-name'}).text.strip()
#         review['rating'] = div.find('span', {'class': 'a-icon-alt'}).text.split(' ')[0]
#         a_tag = div.find('span', {'data-hook': 'review-title'})
#         if a_tag:
#             review['review_title'] = a_tag.find_all('span')[0].text.strip()
#             # review['review_title'] = a_tag.find_all('span')[-1].text.strip()
#         else:
#             review['review_title'] = ""
#         if div.find('span', {'class': 'a-size-base review-text review-text-content'}).span:
#             text = div.find('span', {'class': 'a-size-base review-text review-text-content'}).span.text.strip()
#             if text:
#                 language = detect(text)
#                 if language != 'en':
#                     text = translator.translate(text).text
#         else:
#             text=""
#         review['review_text'] = text
#         review['review_date'] = div.find('span', {'data-hook': 'review-date'}).text.strip()
#         pname = soup.find('a', {'data-hook': 'product-link'}).text.strip()
#         review['product_name'] = re.sub(r',', '', pname)
#         reviews.append(review)
#     print("-------------completed successfully---------------")
#     return reviews

In [10]:
from googletrans import Translator
from langdetect import detect
import re
from bs4 import BeautifulSoup

translator = Translator()

def extract_reviews(soup):
    reviews = []
    review_divs = soup.find_all('div', {'data-hook': 'review'})

    for div in review_divs:
        try:
            review = {}
            review['customer_name'] = div.find('span', {'class': 'a-profile-name'}).text.strip()
            review['rating'] = div.find('span', {'class': 'a-icon-alt'}).text.split(' ')[0]
            
            a_tag = div.find('span', {'data-hook': 'review-title'})
            if a_tag:
                review['review_title'] = a_tag.find_all('span')[0].text.strip()
            else:
                review['review_title'] = ""
            
            text_element = div.find('span', {'class': 'a-size-base review-text review-text-content'})
            if text_element and text_element.span:
                text = text_element.span.text.strip()
                if text:
                    language = detect(text)
                    if language != 'en':
                        text = translator.translate(text, dest='en').text
                review['review_text'] = text
            else:
                review['review_text'] = ""
            
            review['review_date'] = div.find('span', {'data-hook': 'review-date'}).text.strip()
            pname = soup.find('a', {'data-hook': 'product-link'}).text.strip()
            review['product_name'] = re.sub(r',', '', pname)
            reviews.append(review)
        except Exception as e:
            print(f"An error occurred: {e}")
            continue  # Skip to the next review in case of an error

    print("-------------completed successfully---------------")
    return reviews


In [3]:
def save_to_csv(reviews, filename='reviews.csv'):
    df = pd.DataFrame(reviews)
    if os.path.isfile(filename):
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        df.to_csv(filename, index=False)

In [4]:
all_reviews = []

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)  # Wait for the page to load
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_reviews.extend(extract_reviews(soup))

# Read URLs from the file
with open('urls.txt', 'r') as file:
    urls = file.readlines()

# Remove any extra whitespace (e.g., newline characters) from each URL
urls = [url.strip() for url in urls]

# Setup Selenium WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Loop through each URL
for current_url in urls:
    try:
        driver.get(current_url)
        time.sleep(2)  # Wait for the page to load
        if "Sorry! We couldn't find that page" in driver.page_source:
            print(f"Page not found: {current_url}. Stopping execution.")
            break
        else:
            print(f"Scraping reviews from {current_url}")
            scrape_reviews(current_url, driver)
            time.sleep(1)  # Add a delay to avoid hitting the server too frequently
    except Exception as e:
        print(f"An error occurred: {e}")
        break

# Close the driver
driver.quit()


Scraping reviews from https://www.amazon.in/Intel-BX8071512100F-INTEL-I3-12100F-DESKTOP/product-reviews/B09NPJX7PV/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
-------------completed successfully---------------
Scraping reviews from https://www.amazon.in/Intel-BX8071512100F-INTEL-I3-12100F-DESKTOP/product-reviews/B09NPJX7PV/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2
-------------completed successfully---------------
Scraping reviews from https://www.amazon.in/Intel-BX8071512100F-INTEL-I3-12100F-DESKTOP/product-reviews/B09NPJX7PV/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3
-------------completed successfully---------------
Scraping reviews from https://www.amazon.in/Intel-BX8071512100F-INTEL-I3-12100F-DESKTOP/product-reviews/B09NPJX7PV/ref=cm_cr_getr_d_paging_btm_next_4?ie=UTF8&reviewerType=all_reviews&pageNumber=4
-------------completed successfully---------------
Scraping reviews from https://www

In [12]:
len(all_reviews)

100

In [13]:
save_to_csv(all_reviews,"xyz.csv")